# Monkeypox KG

# Package installation is required only once

In [ ]:
#pip install -r packages.txt

In [1]:
import pybel
from tqdm import tqdm
import pandas as p
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from ByCovidGraph import *
import pubchempy as pcp
import urllib
from urllib.parse import urlparse
import io


C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\pybel\struct\operations.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Drugs against monkeypox from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12 + chembl database

In [2]:
mpox_chembl = 'CHEMBL152 CHEMBL203321 CHEMBL1257073 CHEMBL220610 CHEMBL221054 CHEMBL1257073 CHEMBL375768 CHEMBL220150'.split()


# Reading file from pubchem for monkeypox 
# https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

In [5]:

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem = mpox_pchem['cid']
mpox_pchem = list(set(mpox_pchem))


In [6]:
pchem2chembl_list = cid2chembl(mpox_pchem)

Pubchem: 134406 Converted to: CHEMBL321500
Pubchem: 44420748 Converted to: CHEMBL375768
Pubchem: 101879053 Converted to: CHEMBL3357216
Pubchem: 101879054 Converted to: CHEMBL3357215
Pubchem: 16124688 Converted to: CHEMBL1257073
Pubchem: 11243799 Converted to: CHEMBL220150
Pubchem: 155545375 Converted to: CHEMBL4565849
Pubchem: 11773602 Converted to: CHEMBL220610
Pubchem: 73124 Converted to: CHEMBL494759
Pubchem: 37542 Converted to: CHEMBL1643
Pubchem: 44341289 Converted to: CHEMBL114867
Pubchem: 44341811 Converted to: CHEMBL113930
Pubchem: 44341812 Converted to: CHEMBL326139
Pubchem: 118895042 Converted to: CHEMBL3753637
Pubchem: 118895044 Converted to: CHEMBL3753489
Pubchem: 60613 Converted to: CHEMBL152
Pubchem: 11301322 Converted to: CHEMBL220098
Pubchem: 189926 Converted to: CHEMBL112159
Pubchem: 5271015 Converted to: CHEMBL360506
Pubchem: 11452011 Converted to: CHEMBL219691
Pubchem: 11463022 Converted to: CHEMBL221054
Pubchem: 155521397 Converted to: CHEMBL4449833
Pubchem: 4511103

In [9]:
z = set(mpox_chembl).intersection(pchem2chembl_list)
z

{'CHEMBL1257073',
 'CHEMBL152',
 'CHEMBL220150',
 'CHEMBL220610',
 'CHEMBL221054',
 'CHEMBL375768'}

In [3]:
chemblid = mpox_chembl+pchem2chembl_list
len(chemblid)

NameError: name 'pchem2chembl_list' is not defined

In [ ]:
chemblid = set(chemblid)
chemblid = list(chemblid)
len(chemblid)

In [ ]:
chembl2mech = RetMech(chemblid)

In [ ]:
chembl2dis = RetDrugInd(chemblid)

In [ ]:
chembl2act = RetAct(chemblid,0)

In [ ]:
prtn_as_chembl = Ret_chembl_protein(chembl2act) + Ret_chembl_protein(chembl2mech)
prtn_as_chembl = set(prtn_as_chembl)
prtn_as_chembl = list(prtn_as_chembl)
prtn_as_chembl

In [ ]:
chembl2uprot = chembl2uniprot(prtn_as_chembl,0)

In [ ]:
chembl2uprot

In [ ]:
#update chembl protein nodes with gene symbol
#use previously created variable that stores activity info
chembl2act = chembl2gene2path(chembl2uprot,chembl2act)

In [ ]:
chembl2act

In [ ]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [ ]:
chembl2uprot

In [ ]:
def chem2gene2path_rel(named_chem2geneList,itmpGraph):
    for item in named_chem2geneList:
        #print(item)
        itemLen = len(named_chem2geneList[item])-1
        #print(itemLen)
        for j in range(itemLen-1):
            #print(j)
            #print('yo',named_chem2geneList)
            #if 'accession' not in named_chem2geneList[j]:
                #continue
                
            itmpGraph.add_association(MicroRna(namespace='HP', name=named_chem2geneList[item][itemLen]['component_synonym']),
                                      Rna(namespace='Pathway',name=named_chem2geneList[item][j]['xref_name']),
                                      citation='ChEMBL database', evidence='ChEMBL query',
                                      Reactome=named_chem2geneList[item][j]['xref_id'])

    return(itmpGraph)


In [ ]:
mpox_graph = chem2moa_rel_2(chembl2mech,mpox_graph)
mpox_graph = chem2dis_rel(chembl2dis,mpox_graph)
mpox_graph = chem2act_rel_2(chembl2act,mpox_graph)
mpox_graph = chem2gene2path_rel(chembl2uprot,mpox_graph)
to_jupyter(mpox_graph)

In [ ]:
# filename = ('data/normalized_data/monkeypox_pickle2')
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)

In [3]:
#import monkey pox graph
infile = open('data/normalized_data/monkeypox_pickle2','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [4]:
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>

In [ ]:
#Export Uprot ids from the current graph
chemblProt = []
for item in chembl2uprot:
    for j in range(len(chembl2uprot[item])):
        if('accession' in chembl2uprot[item][j]):
            chemblProt.append(chembl2uprot[item][j]['accession'])

chemblProt

In [ ]:
chemblProt = ['O43865', 'P20839', 'Q8JXU8', 'P02545', 'P23526', 'P06856','P08684']

# Working with monkeypox proteins from uniprot

In [10]:
mpox_prot_df = pd.read_excel('data/normalized_data/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx')
mpox_prot= list(mpox_prot_df['Entry'])
mpox_prot

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['Q8V4Y0',
 'Q8V4S4',
 'Q8V571',
 'P04363',
 'Q8V4V3',
 'Q8V4U9',
 'Q8V4T3',
 'Q8V518',
 'Q8V4V4',
 'Q8V4T7',
 'Q8V566']

# Genes from Malacards

In [ ]:
mpox_genes_df = pd.read_excel('data/normalized_data/malacardsGenes.xlsx')

In [ ]:
mpox_genes = list(mpox_genes_df['Entry'])
mpox_genes

In [ ]:
uprots = mpox_prot+mpox_genes+chemblProt

In [ ]:
uprots

In [ ]:
def ExtractFromUniProt(uniprot_id):
    from bioservices import UniProt
    Uniprot_Dict = []
    # Make a link to the UniProt webservice
    service = UniProt()

    for id in uniprot_id:

        # create URL for each uniprot id
        url = 'https://www.uniprot.org/uniprot/' + id + '.txt'
        print(url)

        #Retrieve data for id in text format if found in uniprot
        
        try:
            ret_uprot = urllib.request.urlopen(url)
        except urllib.request.HTTPError:
            #uniprot_id.remove(id)
            continue
        
        
#         with urllib.request.urlopen(url) as response:
#            ret_uprot = response.read()

        print(id)
        id_copy = id
        i = 0
        j = 0
        k = 0
        id = {}
        id['Disease'] = {}
        id['Reactome'] = {}
        id['Function'] = {}
        id['BioProcess'] = {}
        id['Gene'] = {}
        #id['Gene'] = {}
        # print(id)

        # parse each line looking for info about disease, pathway, funcn, bp and so on
        for line in ret_uprot:

            line = line.decode('utf-8')

            # parse lines with disease and extract disease names and omim ids
            if '-!- DISEASE:' in line:
                if ('[MIM:' in line):
                    dis = line.split(':')
                    # dis returns list of splitted text, [1] = name of dis, [2] = OMIM ID, extra chars need cleaning
                    # print(dis[1][1:-5])
                    # print(dis[2][:-1])
                    id['Disease'].update({dis[1][1:-5]: dis[2][:-1]})

            # extract reactome ids and names
            if 'Reactome;' in line:
                ract = line.split(';')
                # ract returns list with reactome id and name, needs cleaning
                id['Reactome'].update({ract[2][1:-2]: ract[1][1:]})
                # print(ract[1][1:])
                # print(ract[2][1:-2])

            # look for functions
            if ' F:' in line:
                if j < 5:
                    # take only first 5 entries for now
                    # print(j)
                    fn = line.split(';')
                    # fn returns list with GO ids and names
                    id['Function'].update({fn[2][3:]: fn[1][1:]})
                    # print(fn[1][1:])
                    # print(fn[2][3:])
                    j += 1

            # look for biological processes
            if ' P:' in line:
                if i < 5:
                    # take only first 5 entries for now
                    # print(i)
                    bp = line.split(';')
                    # bp returns list with GO ids and names
                    id['BioProcess'].update({bp[2][3:]: bp[1][1:]})
                    #print(bp[1][1:])
                    #print(bp[2][3:])
                    i += 1
                    
            if 'GN   Name' in line:
                #print(line)
                
                if k == 0:
                    gene = line.split('=')
                    print(gene)
                    gene = gene[1].split(' ')
                    #print(gene[0])
                    if ';' in gene[0]:
                        gene=gene[0].split(';')
                        #id['Gene'].update({gene[0]})
                        gene = {'Gene': gene[0]}
                        #id.update(gene)
                    else:
                        gene = {'Gene':gene[0]}
                        #id.update(gene)
                    id.update(gene)
                    #print(id['Gene'])    
                    k +=1
            
            #else:
                #id.update({'Gene': ''})
                    
            #print(id['Gene'])
            
        Uniprot_Dict.append(id)

    Uniprot_Dict = dict(zip(uniprot_id, Uniprot_Dict))

    return(Uniprot_Dict)
    

In [ ]:
uprots_ext = ExtractFromUniProt(uprots)

In [ ]:
uprots_ext = ExtractFromUniProt(['Q8V4Y0','Q8V4S4','P01562','P01730'])

In [ ]:
uprots_ext

In [ ]:
#a = 'GN   Name=B4R {ECO:0000312|EMBL:AAL40623.1};'
a = 'GN   Name=IFNA1;'
a = a.split('=')
a = a[1].split(" ")
a[0]


In [ ]:
if ';' in a[0]:
    a=a[0].split(";")
    


In [ ]:
a[0]

In [ ]:
def uniprot_rel(named_uprotList,itmpGraph):
    
    for item in named_uprotList:
        #print(named_uprotList[item]['Function'].keys())
        fun=list(named_uprotList[item]['Function'].keys())
        bp = list(named_uprotList[item]['BioProcess'].keys())
        for f in fun:
            if str(named_uprotList[item]['Gene']) != 'nan' and not isinstance(named_uprotList[item]['Gene'],dict) :
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
                
        
        for b in bp:
            if str(named_uprotList[item]['Gene']) != 'nan' and not isinstance(named_uprotList[item]['Gene'],dict):
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),BiologicalProcess(namespace='BioPro',name=b),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=b),
                                         citation='UniProt database',evidence='UniProt query')
        
    return(itmpGraph)
        

In [ ]:
mpox_graph = uniprot_rel(uprots_ext,mpox_graph)
to_jupyter(mpox_graph)

In [ ]:
# filename = 'data/normalized_data/monkeypox_pickle'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)

In [5]:
#import monkey pox graph
infile = open('data/normalized_data/monkeypox_pickle','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [6]:
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>

In [ ]:

def _get_target_data(protein_list: list, organism: str):
    """Get chemical for target data from ChEMBL"""
    df_data = []

    target = new_client.target
    activity = new_client.activity

    for protein in protein_list:
        if pd.isna(protein):
            continue
        try:
            prot_data = [target.search(protein)[0]]

            # Search for protein with same synonym
            if prot_data == [None]:
                prot_data = target.filter(
                    target_synonym__icontains=protein, target_organism__istartswith=organism
                ).only(['target_chembl_id', 'target_pref_name', 'molecule_chembl_id', 'molecule_pref_name'])
        except HttpBadRequest:
            print(f'Cannot search for {protein} due to chembl error')
            continue

        # No results found
        if not prot_data:
            continue

        for prot in tqdm(prot_data, f'Analying data for {protein}'):
            # Absence of chembl id
            if not prot['target_chembl_id']:
                continue

            prot_activity_data = activity.filter(
                target_chembl_id=prot['target_chembl_id'],
                assay_type_iregex='(B|F)',
            ).only([
                'pchembl_value', 'molecule_chembl_id', 'activity_id', 'target_pref_name', 'molecule_pref_name'
            ])

            if len(prot_activity_data) < 1:
                continue

            for i in prot_activity_data:
                tmp = {}

                if i['pchembl_value'] is None:
                    continue

                pchembl_val = i['pchembl_value']

                if float(pchembl_val) < 6:
                    tmp['activity'] = 'inhibitor'
                else:
                    tmp['activity'] = 'activator'

                tmp['protein_symbol'] = protein
                tmp['protein_name'] = i['target_pref_name']
                tmp['aid'] = str(i['activity_id'])
                tmp['chembl_id'] = i['molecule_chembl_id']
                tmp['compound_name'] = i['molecule_pref_name'].capitalize() if i['molecule_pref_name'] else ''
                df_data.append(tmp)

    # Merge duplicated protein-chemical entries into one
    df = pd.DataFrame()

    for idx, row in tqdm(enumerate(df_data), total=len(df_data), desc='Preparing data'):
        if idx == 0:
            df = df.append(row, ignore_index=True)
        else:
            _in_df = df.loc[
                (df['protein_symbol'] == row['protein_symbol']) & (df['chembl_id'] == row['chembl_id'])
            ]

            if _in_df.empty:
                df = df.append(row, ignore_index=True)
            else:
                row_index = _in_df.index

                # Check existing citations
                existing_assays = set(df.loc[row_index, 'aid'].values[0].split(' | '))
                old_count = len(existing_assays)
                existing_assays.add(row['aid'])
                new_count = len(existing_assays)

                # Check if new citation added, if yes - add respective data
                if old_count < new_count:
                    df.loc[row_index, 'aid'] = ' | '.join(existing_assays)
    df = df[['activity', 'protein_symbol', 'protein_name', 'aid', 'chembl_id', 'compound_name']]
    return df


In [ ]:
def target_list_to_chemical(
    proteins: list,
    organism: str = 'Homo sapiens',
    output_dir: str = ''
) -> None:
    """Extract chemical information on list of targets
    Usage:
    >> target_list_to_chemical(proteins=['RIPK'])
    """

    df = _get_target_data(protein_list=proteins, organism=organism)
    #os.makedirs(output_dir, exist_ok=True)
    #df.to_csv(os.path.join(output_dir, 'chemical_annotated.csv'), sep='\t', index=False)
    return(df)


In [ ]:
uprot2chem = target_list_to_chemical(uprots)

In [ ]:
uprot2chem

In [ ]:
# filename = 'data/normalized_data/uprot2chembl'
# outfile = open(filename,'wb')
# pickle.dump(uprot2chem,outfile)

In [7]:
infile = open('data/normalized_data/uprot2chembl','rb')
uprot2chem = pickle.load(infile)
infile.close()

In [8]:
uprot2chem

,activity,protein_symbol,protein_name,aid,chembl_id,compound_name
0,activator,P01730,T-cell surface antigen CD4,105411,CHEMBL506605,Isocomplestatin
1,activator,P01730,T-cell surface antigen CD4,106547,CHEMBL525803,Chloropeptin
2,activator,P01730,T-cell surface antigen CD4,106550,CHEMBL3143464,
3,inhibitor,P01730,T-cell surface antigen CD4,346232,CHEMBL3138360,
4,inhibitor,P01730,T-cell surface antigen CD4,346237,CHEMBL3138345,
...,...,...,...,...,...,...
53593,inhibitor,P08684,Cytochrome P450 3A4,22926906,CHEMBL4791586,
53594,inhibitor,P08684,Cytochrome P450 3A4,22953965,CHEMBL4649457,Bms-986166
53595,inhibitor,P08684,Cytochrome P450 3A4,22982169,CHEMBL4784058,
53596,inhibitor,P08684,Cytochrome P450 3A4,22991278 | 22991276,CHEMBL4787795,


In [9]:
active = uprot2chem.loc[uprot2chem['activity'].str.contains('activator',na=False)]
active = active.reset_index(drop=True)

In [ ]:
active

In [ ]:
active['compound_name'][3] == ''

In [10]:
active = active[active['compound_name'] != '']
active = active.reset_index(drop=True)

In [11]:
chem = active['chembl_id']

In [12]:
len(chem)

579

In [ ]:
chem = ['CHEMBL506605',
 'CHEMBL525803',
 'CHEMBL3143464',
 'CHEMBL3138360',
 'CHEMBL3138345',
 'CHEMBL3138187',
 'CHEMBL3138374',
 'CHEMBL3138095',
 'CHEMBL3138378',
 'CHEMBL3138119',
 'CHEMBL231224',
 'CHEMBL267655',
 'CHEMBL388209',
 'CHEMBL230799',
 'CHEMBL267658',
 'CHEMBL388208',
 'CHEMBL267657',
 'CHEMBL230798',
 'CHEMBL388207',
 'CHEMBL230796',
 'CHEMBL1852155',
 'CHEMBL1819181',
 'CHEMBL4783030',
 'CHEMBL4755141',
 'CHEMBL4783761',
 'CHEMBL4750160',
 'CHEMBL4743003',
 'CHEMBL4782080',
 'CHEMBL4746252',
 'CHEMBL4747494',
 'CHEMBL4758659',
 'CHEMBL4750983',
 'CHEMBL4786809',
 'CHEMBL4748787',
 'CHEMBL4785236',
 'CHEMBL4791666',
 'CHEMBL4525466',
 'CHEMBL4471585']

In [28]:
chem2dis = RetDrugInd(chem)

[]
[]
[]
[{'mesh_heading': 'Graft vs Host Disease'}, {'mesh_heading': 'Immune System Diseases'}, {'mesh_heading': 'Renal Insufficiency, Chronic'}, {'mesh_heading': 'Leukemia, Myeloid, Acute'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[{'mesh_heading': 'Hepatitis C'}]
[{'mesh_heading': 'Hepatitis C'}, {'mesh_heading': 'Hepatitis C, Chronic'}]


KeyboardInterrupt: 

In [ ]:
chem2mech = RetMech(chem)

In [ ]:
chem2act = RetAct(chem,0)

In [ ]:
chembl2actmechdis = {}
chembl2actmechdis['activity'] = chem2act
chembl2actmechdis['mechanism'] = chem2mech
chembl2actmechdis['disease']= chem2dis

In [ ]:
# filename = 'data/normalized_data/chembl2actmechdis'
# outfile = open(filename,'wb')
# pickle.dump(chembl2actmechdis,outfile)

In [13]:
infile = open('data/normalized_data/chembl2actmechdis','rb')
chembl2actmechdis = pickle.load(infile)
infile.close()

In [14]:
chem2act = chembl2actmechdis['activity']
chem2mech = chembl2actmechdis['mechanism']
chem2dis = chembl2actmechdis['disease']

In [15]:
chemid2 = Ret_chembl_protein(chem2act) + Ret_chembl_protein(chem2mech)
chemid2

['CHEMBL325',
 'CHEMBL614067',
 'CHEMBL614530',
 'CHEMBL262',
 'CHEMBL4641',
 'CHEMBL614997',
 'CHEMBL3437',
 'CHEMBL613107',
 'CHEMBL612707',
 'CHEMBL2675',
 'CHEMBL614696',
 'CHEMBL249',
 'CHEMBL614056',
 'CHEMBL4234',
 'CHEMBL1941',
 'CHEMBL287',
 'CHEMBL4398',
 'CHEMBL2842',
 'CHEMBL613996',
 'CHEMBL1824',
 'CHEMBL2327',
 'CHEMBL275',
 'CHEMBL1641347',
 'CHEMBL5619',
 'CHEMBL614149',
 'CHEMBL224',
 'CHEMBL1898',
 'CHEMBL395',
 'CHEMBL2002',
 'CHEMBL254',
 'CHEMBL6007',
 'CHEMBL1293232',
 'CHEMBL1791',
 'CHEMBL340',
 'CHEMBL2664',
 'CHEMBL2056',
 'CHEMBL2071',
 'CHEMBL398',
 'CHEMBL3227',
 'CHEMBL1293224',
 'CHEMBL1697668',
 'CHEMBL1293267',
 'CHEMBL614649',
 'CHEMBL4295974',
 'CHEMBL222',
 'CHEMBL2111457',
 'CHEMBL223',
 'CHEMBL1293235',
 'CHEMBL2095158',
 'CHEMBL614071',
 'CHEMBL392',
 'CHEMBL1827',
 'CHEMBL1907588',
 'CHEMBL3629',
 'CHEMBL613835',
 'CHEMBL614515',
 'CHEMBL2094257',
 'CHEMBL331',
 'CHEMBL2097163',
 'CHEMBL2095173',
 'CHEMBL2094256',
 'CHEMBL1293237',
 'CHEMBL2039'

In [16]:
chem2uni = chembl2uniprot(chemid2,0)

old 372
newLen 367
5


In [17]:
chem2uni

{'CHEMBL325': [{'xref_id': 'R-HSA-1362277',
   'xref_name': 'Transcription of E2F targets under negative control by DREAM complex.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1362300',
   'xref_name': 'Transcription of E2F targets under negative control by p107 (RBL1) and p130 (RBL2) in complex with HDAC1.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1538133',
   'xref_name': 'G0 and Early G1.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-193670',
   'xref_name': 'p75NTR negatively regulates cell cycle via SC1.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-201722',
   'xref_name': 'Formation of the beta-catenin:TCF transactivating complex.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2122947',
   'xref_name': 'NOTCH1 Intracellular Domain Regulates Transcription.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2173795',
   'xref_name': 'Downregulation of SMAD2/3:SMAD4 transcriptional activity.',
   'xref_src_db': 'Reactome'},
  {'xref_id

In [ ]:
protList = []
for key in chem2uni.keys():
    #print(chem2uni[key])
    for item in chem2uni[key]:
        if 'accession' in item:
            protList.append(item['accession'])
            #print(item['accession'])

In [ ]:
len(protList)

In [ ]:
protList_ext = ExtractFromUniProt(protList[:100])

In [ ]:
protList_ext_1 = ExtractFromUniProt(protList[100:200])

In [ ]:
protList_ext_2 = ExtractFromUniProt(protList[200:310])

In [ ]:
allprot = protList_ext | protList_ext_1 | protList_ext_2

In [ ]:
allprot

In [ ]:
# filename = 'data/normalized_data/allprot_fromchembl'
# outfile = open(filename,'wb')
# pickle.dump(allprot,outfile)

In [18]:
infile = open('data/normalized_data/allprot_fromchembl','rb')
allprot = pickle.load(infile)
infile.close()

In [10]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [21]:
mpox_graph = uniprot_rel(allprot,mpox_graph)
#to_jupyter(mpox_graph)

In [22]:
pchem_act_new= chembl2gene2path(chem2uni,chem2act)

In [ ]:
chem2uni

In [24]:
def chem2gene2path_rel(named_chem2geneList,itmpGraph):
    for item in named_chem2geneList:
        #print(item)
        itemLen = len(named_chem2geneList[item])-1
        #print(itemLen)
        for j in range(itemLen-1):
            #print(named_chem2geneList)
            itmpGraph.add_association(MicroRna(namespace='HP', name=named_chem2geneList[item][itemLen]['component_synonym']),
                                      Rna(namespace='Pathway',name=named_chem2geneList[item][j]['xref_name']),
                                      citation='ChEMBL database', evidence='ChEMBL query',
                                      Reactome=named_chem2geneList[item][j]['xref_id'])

    return(itmpGraph)

In [25]:
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
#to_jupyter(mpox_graph)

In [ ]:
chem2uni['CHEMBL2094125'][2]['component_synonym']

In [32]:
# filename = 'data/normalized_data/monkeypox_pickle_final_12thJuly'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)
# outfile.close()

In [34]:
infile = open('data/normalized_data/monkeypox_pickle_final_12thJuly','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [35]:
mpox_graph.summarize()

---------------------  ---------------
Name                   Monkeypox Graph
Version
Number of Nodes        4175
Number of Namespaces   9
Number of Edges        13320
Number of Annotations  0
Number of Citations    2
Number of Authors      0
Number of Components   10
Number of Warnings     0
Network Density        7.64E-04
---------------------  ---------------

Type (7)             Count  Example
-----------------  -------  ----------------------------------------------------
miRNA                  888  m(Function:"phosphoprotein binding")
Abundance              862  a(ChEMBLAssay:CHEMBL748128)
BiologicalProcess      775  bp(BioPro:"activation of protein kinase B activity")
RNA                    751  r(Pathway:"Signaling by ERBB2 KD Mutants.")
Pathology              558  path(ChEMBL:CHEMBL1256818)
Protein                305  p(MP:HSD17B3)
Population              36  pop(Disease:"Liver Cirrhosis")

Prefix (9)    Name                            Count  Example
------------  -----------